In [3]:
import pandas as pd
import random
import numpy as np

## Synthetically creating our data to experiment

In [19]:
random.seed(1)

def _fillHighValues(x):
    return random.randint(8,10)

def _fillMidValues(x):
    return random.randint(4,7)

def _fillLowValues(x):
    return random.randint(1,3)

def _fillNoise(x):
    return random.randint(1,10)

def _engagedHigh(x):
    return random.randint(4,5)

def _engagedMid(x):
    return random.randint(2,3)

def _engagedLow(x):
    return random.randint(1,1)

def _engagedNoise(x):
    return random.randint(1,5)

### Metrics (1 - bad, 10 - good)
Emotion - Mood gradient , normalized from 1 to 10

Headgaze - (RA Evidence) focus on front face (rather than side gaze) - normalized from 1 to 10

Motion - (RA Evidence) amount of motion over a span of 10 minutes - normalized from 1 to 10

handPose - number of times hand raised over a span of 10 minutes - normalized from score of 1 to 10

sleepPose - number of times in sleeping pose over a span of 10 minutes - normalized from a score of 1(good) to 10(bad)

## Correlation of metrics
Emotion - Moderate-Highcorrelation with engagement, behaviors and attentiveness

HeadGaze - Moderate-High Correlation, facing front fairly justify attentivness, looking away is obvious disengagement

Motion - Low-Moderate Correlation, increasing micro-motions shows correlation with attentivness (Reseach Article)

HandPose - High Correlation, raising hands definitely engagement with current session 

SleepPose - High correlation, detected = not engaged

Correlation ranking
1. HandPose
2. SleepPose
3. Emotion
4. HeadGaze
5. Motion

### Synthetic Data
1. We work with 3000 training examples with this neural network
2. Assuming 10% noise in data, we work with 2700 reliable datasets, 300 noise datsets (random labels)
3. Breaking down the 2700 training examples

##### we break into high, middle and low tier ratings with a few simple rules in place to test out
1. high - if metrics 1,3,4,5 is high and 2 (sleepPose) is low, return high value
2. mid - if metrics 1,2,3,4,5 is moderate, return a moderate value
3. low - if mtrics 1,3,4,5 is low and 2 (sleepPose) is low, return low value
4. noise - all are noise
5. engagement - we will take softmax of range 1 to 5, 4-5 being most engaged, 2-3 being mildly engaged, 1 least engaged

In [20]:
df = pd.DataFrame(index = range(0,3000), columns = ['Emotion', 'HeadGaze', 'Motion', 'handPose', 'sleepPose', 'Engagement'])

In [21]:
#First 80 rows are synthetically created, last 20 rows will be noise

df['Emotion'][:900] = df['Emotion'][:900].apply(lambda x : _fillHighValues(x))
df['Emotion'][900:1800] = df['Emotion'][900:1800].apply(lambda x : _fillMidValues(x))
df['Emotion'][1800:2700] = df['Emotion'][1800:2700].apply(lambda x : _fillLowValues(x))
df['Emotion'][2700:] = df['Emotion'][2700:].apply(lambda x : _fillNoise(x))

df['HeadGaze'][:900] = df['HeadGaze'][:900].apply(lambda x : _fillHighValues(x))
df['HeadGaze'][900:1800] = df['HeadGaze'][900:1800].apply(lambda x : _fillMidValues(x))
df['HeadGaze'][1800:2700] = df['HeadGaze'][1800:2700].apply(lambda x : _fillLowValues(x))
df['HeadGaze'][2700:] = df['HeadGaze'][2700:].apply(lambda x : _fillNoise(x))

df['Motion'][:900] = df['Motion'][:900].apply(lambda x: _fillHighValues(x))
df['Motion'][900:1800] = df['Motion'][900:1800].apply(lambda x: _fillMidValues(x))
df['Motion'][1800:2700] = df['Motion'][1800:2700].apply(lambda x: _fillLowValues(x))
df['Motion'][2700:] = df['Motion'][2700:].apply(lambda x: _fillNoise(x))

df['handPose'][:900] = df['handPose'][:900].apply(lambda x: _fillHighValues(x))
df['handPose'][900:1800] = df['handPose'][900:1800].apply(lambda x: _fillMidValues(x))
df['handPose'][1800:2700] = df['handPose'][1800:2700].apply(lambda x: _fillLowValues(x))
df['handPose'][2700:] = df['handPose'][2700:].apply(lambda x: _fillNoise(x))

df['sleepPose'][:900] = df['sleepPose'][:900].apply(lambda x: _fillLowValues(x))
df['sleepPose'][900:1800] = df['sleepPose'][900:1800].apply(lambda x: _fillMidValues(x))
df['sleepPose'][1800:2700] = df['sleepPose'][1800:2700].apply(lambda x: _fillHighValues(x))
df['sleepPose'][2700:] = df['sleepPose'][2700:].apply(lambda x: _fillNoise(x))

df['Engagement'][:900] = df['Engagement'][:900].apply(lambda x : _engagedHigh(x))
df['Engagement'][900:1800] = df['Engagement'][900:1800].apply(lambda x : _engagedMid(x))
df['Engagement'][1800:2700] = df['Engagement'][1800:2700].apply(lambda x : _engagedLow(x))
df['Engagement'][2700:] = df['Engagement'][2700:].apply(lambda x : _engagedNoise(x))

C:\Users\Gabriel Sze\Anaconda3\envs\tensorlearn\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Gabriel Sze\Anaconda3\envs\tensorlearn\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Gabriel Sze\Anaconda3\envs\tensorlearn\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff

In [22]:
df.describe()

,Emotion,HeadGaze,Motion,handPose,sleepPose,Engagement
count,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,5.517667,5.498000,5.482667,5.461667,5.505000,2.679667
std,3.018619,2.987357,2.988089,2.985996,3.007156,1.472563
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,3.000000,3.000000,3.000000,3.000000,3.000000,1.000000
50%,6.000000,6.000000,5.000000,5.000000,6.000000,3.000000
75%,8.000000,8.000000,8.000000,8.000000,8.000000,4.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,5.000000


In [23]:
df.head()

,Emotion,HeadGaze,Motion,handPose,sleepPose,Engagement
0,8,10,8,9,3,5
1,10,8,8,8,3,4
2,8,8,10,8,1,5
3,9,8,8,10,3,5
4,8,8,10,8,2,5


In [24]:
df.to_excel('EngagementTest3.xlsx')